# Live Mount Demo

## Look at the environment before the Live Mount

In this section we want to look at the current list of databases and the size of each database. This way we can understand what is out there currently before we make a change and create a Live Mount. Looking at the left hand side of your screen you will see the SQL Server you are connect to and the list of databases under the database folder. The below shows you the database its size, which you cannot easily get from the left hand side.

In [ ]:
-- List database and its size
WITH fs
AS
(
    SELECT database_id, type, size * 8.0 / 1024 size
    FROM sys.master_files
)
SELECT name
    , (SELECT SUM(size) FROM fs WHERE type = 0 AND fs.database_id = db.database_id) DataFileSizeInMB
    , (SELECT SUM(size) FROM fs WHERE type = 1 AND fs.database_id = db.database_id) LogFileSizeInMB
FROM sys.databases db
WHERE db.database_id > 4
ORDER BY name

## Live Mount a Database

A database of any size can be Live Mounted, but for this demo, take the largest database, which will either be named Cork or PaloAlto.

To create the Live Mount, you will need to log into **Rubrik Security Cloud** (RSC). You can follow these steps to create your Live Mount. 

[Rubrik Recovery Option 3: Live Mount](https://rubrik.atlassian.net/wiki/spaces/SEK/pages/2472444754/Recovery+Option+3%3A+Live+Mount)

## Look at the environment after the Live Mount

In this section we want to look at the new list of databases and the size of each database. Now we can see that a Live Mounted Database is there and how large it is.

In [ ]:
-- List database and its size
WITH fs
AS
(
    SELECT database_id, type, size * 8.0 / 1024 size
    FROM sys.master_files
)
SELECT name
    , (SELECT SUM(size) FROM fs WHERE type = 0 AND fs.database_id = db.database_id) DataFileSizeInMB
    , (SELECT SUM(size) FROM fs WHERE type = 1 AND fs.database_id = db.database_id) LogFileSizeInMB
FROM sys.databases db
WHERE db.database_id > 4
ORDER BY name

## Show the Location of the Database Files

The below section shows where all of the files are located for each database. As you can see, "normal" databases are located on the C drive on mount points. A Live Mount database is located on a SMB3 share located on the Rubrik storage. 

A Live Mounted database is a fully functional database. Any read or write operation that you can do against any other database on the SQL Server can be done against one that is Live Mounted. The only difference between a Live Mounted database and a "normal" database is the storage it sits on. A "normal" database sits on your attached storage allocated from your SAN, while a Live Mounted database is on Rubrik storage attached via a secure SMB3 share.

In [ ]:
-- Show the Location of the Database Files
SELECT DB_NAME(database_id) as 'Database Name'
    , type_desc as 'File Type'
    , name as 'Logical Name'
    , physical_name as 'Physical Name'
    , state_desc
    , size * 8.0 / 1024 as 'File in MB'
FROM sys.master_files
WHERE database_id > 4
ORDER BY database_id

## Unmount the Live Mounted Database

You have seen that a database can be made available from a backup in a very short time frame. This database is fully functional and read write. When you are done with it, you will want to unmount it from the Rubrik UI. This unmounting process will remove the database from the SQL Server, remove the SMB3 share off of the Rubrik storage, and delete all changes that were made while the database was Live Mounted. 

# Live Mount Use Cases

The performance profile of a Live Mounted database sitting on Rubrik storage will be different than a database sitting on storage allocated from your storage array in your datacenter. A Live Mounted database will not provide the same level query performance that a database sitting on your storage array will. What this means is that you will not run performance workloads against a Live Mounted database. However there are plenty of places you can use Live Mount effectively.

- **DBA Backup Validation Testing:** No matter the product you trust to do your backup, if you do not test your backup, you do not have a backup. The way DBAs will often test a backup is by restoring the backup to some other server. This requires resources that are not always available to a DBA. A MSSQL Live Mount IS a database restore. Rubrik is rehydrating the database and applying transaction log backups to provide a fully functional, read/write database. If you can Live Mount your backup, you just proved the backup is good and if you need it in the future, you can get it. 
- **Object Level Recovery:** Often is the case where a mistake is done inside of a table. To recover from that mistake, DBAs need to restore from a backup. This often requires additional storage, which requires paperwork, negotiations with the Storage Team, and Server Team needs to add the additional storage for a temporary use, and then time to restore the backup. Then to find out, the backup chosen is not correct and you have to restart the process all over again. Live Mount allows for the database to be up and running in minutes, without the need for additional storage, or configuration on the server team. If you need to iterate, you can tear it down and retry from a different point in time quickly. Once you find your correct backup, recover the object that was broken.
- **Developer Testing:** Because a Live Mount does not use any storage on the target SQL Server, you can give a developer a copy of a databae so they can do functional testing or investigate a problem with their application. 
- **DevOps Automation:** Every DBA will take a backup before they roll out a change. Now they can Live Mount that backup, run their change against the Live Mount and see if the change will work. If it doesn’t they can tear it down and iterate on the failure all without using additional storage not the SQL Server and also preventing an outage to have to take longer than necessary because they proved their change would work before they did the change. 
- **Database Migration Application Smoke Test Validation:** Microsoft is coming out with new versions of SQL Server every couple of years. To keep up is a daunting task. With Live Mount, you can quickly recover a databaes to a new SQL Server with that new version of SQL and see what will break in your application before you do the migration. This allows for DBAs and developers to more quickly iterate on the migration process planning.